# Train your Reasoning Model using NeMo 2.0

This tutorial shows how to fine-tune Meta’s LLaMA 3–8B Instruct model using NVIDIA NeMo and supervised fine-tuning (SFT). You'll train the model on complex instruction-following and reasoning tasks using the[Llama-Nemotron-Post-Training-Data](https://huggingface.co/datasets/nvidia/Llama-Nemotron-Post-Training-Dataset)

### ✅ What You'll Learn
1. Load and preprocess a reasoning-focused instruction dataset.
2. Apply SFT with NeMo 2.0.
3. Train using NeMo's distributed, mixed-precision trainer.
4. Save a fine-tuned checkpoint ready for evaluation or deployment.

### 🚀 Ideal For
1. Multi-turn reasoning (e.g., chain-of-thought)
2. Domain-specific instruction following
3. Question answering, dialogue systems, and agentic behaviors


### NeMo Tools and Resources
[NeMo Framework](https://docs.nvidia.com/nemo-framework/user-guide/latest/overview.html)

### Software Requirements
* Access to latest NeMo Framework NGC Containers
* This playbook has been tested on: nvcr.io/nvidia/nemo:25.02. It is expected to work similarly on other environments.


#### Launch the NeMo Framework container as follows:
```docker run -it -p 8080:8080 -p 8088:8088 --rm --gpus '"device=0,1,2,3"' --ipc=host --network host -v $(pwd):/workspace nvcr.io/nvidia/nemo:25.02```

#### Launch Jupyter Notebook as follows:
```jupyter notebook --allow-root --ip 0.0.0.0 --port 8088 --no-browser --NotebookApp.token=''```

### Hardware Requirements
This playbook has been tested on 4xA100 80G but can be scaled to multiple GPUs as well as multiple nodes by modifying the appropriate parameters


## Step 1. Convert HuggingFace Checkpoint to NeMo Format

Before training, we need to convert the HuggingFace LLaMA 3–8B Instruct checkpoint into NeMo format. NeMo provides a built-in utility ```llm.import_ckpt()``` to handle this conversion.

### ⚠️ This step only needs to be run once per model.
After conversion, the model can be loaded and fine-tuned using NeMo APIs directly.

In [ ]:
import nemo_run as run
from nemo import lightning as nl
from nemo.collections import llm
from megatron.core.optimizer import OptimizerConfig

import torch
import pytorch_lightning as pl
from pathlib import Path
from nemo.collections.llm.recipes.precision.mixed_precision import bf16_mixed
from nemo.lightning.pytorch.optim import CosineAnnealingScheduler, MegatronOptimizerModule, PytorchOptimizerModule
from datetime import datetime

# Configure the import from HuggingFace format to NeMo format
def configure_checkpoint_conversion():
    return run.Partial(
        llm.import_ckpt,
        model=llm.llama3_8b.model(),  # Predefined LLaMA 3 8B model structure
        source="hf:///workspace/Meta-Llama-3-8B-Instruct",  # Path to HF checkpoint (local or HF hub)
        overwrite=False,  # Set to True if you want to overwrite an existing NeMo checkpoint
    )

# Create the configured import task
import_ckpt = configure_checkpoint_conversion()

# Define the local executor (single-node)
local_executor = run.LocalExecutor()

# Execute the checkpoint conversion
run.run(import_ckpt, executor=local_executor)

✓ Checkpoint imported to /root/.cache/nemo/models/Meta-Llama-3-8B

## Step 2. Prepare Data

In this section, we define the configuration for loading and preprocessing an instruction-tuning dataset using NeMo’s FineTuningDataModule. The dataset is expected to be in a structured format (e.g. JSONL), stored locally as ```training.jsonl```.

The training-related parameters like batch size, number of workers, memory mapping, and device count can be modified based on the size of the model, dataset size and compute resources available.

In [ ]:
import json
import shutil
from pathlib import Path
from typing import TYPE_CHECKING, Any, Dict, List, Optional

from datasets import Dataset, DatasetDict, load_dataset

from nemo.collections.llm.gpt.data.core import get_dataset_root
from nemo.collections.llm.gpt.data.fine_tuning import FineTuningDataModule
from nemo.core.config import hydra_runner
from nemo.collections import llm
from nemo.lightning.io.mixin import IOMixin
from nemo.utils import logging

N_DEVICES = 4
timestamp = datetime.now().strftime("%Y%m%d-%H%M")
experiment_name = "baseline-8GPUs-all-data-cleaned-shuffle-no-distrib-sampler-500k-2-workers"

# Define fine-tuning dataset configuration
finetune_config = run.Config(
    llm.FineTuningDataModule,
    dataset_root="/workspace",       # Path to your preprocessed dataset (JSONL, etc.)
    seq_length=8192,                 # Max sequence length for input tokens
    micro_batch_size=1,              # Per-device batch size
    global_batch_size=256,           # Total batch size across all devices
    seed=1234,                       # Seed for reproducibility
    memmap_workers=1,                # Use memory-mapped dataset format for performance
    num_workers=8,                   # DataLoader worker threads
    pin_memory=True,                 # Optimize data transfer to GPU
)

## Step 3. Configure SFT with the NeMo 2.0 API

In this step, we'll use the modular NeMo 2.0 API to configure:

* The distributed trainer

* Logging and checkpointing

* Optimizer with cosine annealing scheduler

* Model definition and resume behavior

* Final recipe assembly for fine-tuning

### ⚙️ 3.1 Configure the Trainer
We define the training strategy with Megatron's Distributed Training strategy using tensor model parallelism and enabling mixed precision with bf16.

In [ ]:
def trainer() -> run.Config[nl.Trainer]:
    strategy = run.Config(
        nl.MegatronStrategy,
        tensor_model_parallel_size=4,
        optimizer_cpu_offload=True
    )
    trainer = run.Config(
        nl.Trainer,
        devices=4,
        num_nodes=1,
        max_steps=100,
        accelerator="gpu",
        strategy=strategy,
        plugins=bf16_mixed(),
        log_every_n_steps=50,
        limit_val_batches=0,
        val_check_interval=0,
        num_sanity_val_steps=0,
        use_distributed_sampler=False,
    )
    return trainer    

### 📝 3.2 Configure Logging and Checkpointing
Logs metrics and periodically saves model checkpoints during training.

In [ ]:
def logger() -> run.Config[nl.NeMoLogger]:
    ckpt = run.Config(
        nl.ModelCheckpoint,
        save_last=True,
        every_n_train_steps=10,
        monitor="reduced_train_loss",
        save_top_k=1,
        save_on_train_epoch_end=True,
        save_optim_on_train_end=True,
    )

    return run.Config(
        nl.NeMoLogger,
        name=f"trained-model-checkpoints",
        log_dir=f"./results-{timestamp}-{N_DEVICES}-devices-{experiment_name}",
        use_datetime_version=True,
        ckpt=ckpt,
        wandb=None
    )

### 📈 3.3 Configure Optimizer with Cosine Annealing
Uses the Adam optimizer with gradient clipping, distributed optimizer support, and a cosine annealing learning rate schedule.

In [ ]:
from megatron.core.optimizer import OptimizerConfig

def lr_scheduler():
    return run.Config(
        CosineAnnealingScheduler,
        warmup_steps=100,        
        constant_steps=1000,
        min_lr=1e-6,
    )
    
def adam_with_cosine_annealing() -> run.Config[nl.OptimizerModule]:
    opt_cfg = run.Config(
        OptimizerConfig,
        optimizer="adam",
        lr=1e-4,
        weight_decay=0.001,
        use_distributed_optimizer=True,
        clip_grad=1.0,
        bf16=True,
    )
    
    return run.Config(
        nl.MegatronOptimizerModule,
        config=opt_cfg,
        lr_scheduler=lr_scheduler(), 
    )

### 🧠 3.4 Define the Base Model and Resume Logic
We use the built-in LLaMA 3 8B config from NeMo and optionally resume from a previously saved checkpoint.

In [ ]:
def llama3_8b() -> run.Config[pl.LightningModule]:
    return run.Config(llm.LlamaModel, config=run.Config(llm.Llama3Config8B))

def resume() -> run.Config[nl.AutoResume]:
    return run.Config(
        nl.AutoResume,
        restore_config=run.Config(
            nl.RestoreConfig,
            path="nemo://Meta-Llama-3-8B-Instruct",  # Change to local path if needed
        ),
        resume_if_exists=True,
    )


### 📦 3.5 Assemble the Fine-Tuning Recipe
This ties together the model, trainer, dataset config, optimizer, and logger into a single training recipe using NeMo’s run.Partial system.

In [ ]:
def configure_finetuning_recipe():
    return run.Partial(
        llm.finetune,
        model=llama3_8b(),
        trainer=trainer(),
        data=finetune_config,  # From earlier step
        log=logger(),
        optim=adam_with_cosine_annealing(),
        resume=resume(),
    )

## ▶️ Step 4: Run Supervised Fine-Tuning (SFT) with NeMo 2.0 and nemo-run
Now that everything is configured (model, trainer, optimizer, logging, and data), it's time to launch the training job using nemo-run's LocalExecutor.

This will:

* Use torchrun to launch a multi-GPU job

* Set environment variables for optimized NCCL behavior

* Kick off the training loop with your full configuration

In [ ]:
def local_executor_torchrun(nodes: int = 1, devices: int = 4) -> run.LocalExecutor:
    # Environment variables to optimize distributed training
    env_vars = {
        "TORCH_NCCL_AVOID_RECORD_STREAMS": "1",
        "NCCL_NVLS_ENABLE": "0",
    }

    return run.LocalExecutor(
        ntasks_per_node=devices,
        launcher="torchrun",
        env_vars=env_vars,
    )

# Execute the training run
if __name__ == '__main__':
    run.run(
        configure_finetuning_recipe(),
        executor=local_executor_torchrun()
    )

In [ ]:
!nvidia-smi

## 🎉 Tada! You Just Trained Your First Reasoning Model!
Congratulations — you've successfully fine-tuned LLaMA 3–8B Instruct into a domain-adapted reasoning model using NVIDIA NeMo 2.0!

Your model is now ready to:

* Answer questions more effectively
* Follow domain-specific instructions
* Support chain-of-thought reasoning in real-world applications

### 🚀 Next Steps
* 🧪 Evaluate your model on reasoning benchmarks (e.g., MMLU, GSM8K)
* 🪄 Add LoRA or QLoRA for even more efficient adaptation
* ☁️ Package the model for deployment or inference with Triton or vLLM
* 📤 Optionally, upload it to HuggingFace or NGC to share with the world